<a href="https://colab.research.google.com/github/Andres8bit/parallel-computing/blob/main/Sparse_Matrix_Multiplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-azp89doy
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-azp89doy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=193f3673c0905a9950367681faab27193128a6e9de84adf53c18aaa115175340
  Stored in directory: /tmp/pip-ephem-wheel-cache-e804_10f/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
# The following cells implement sparce matrix multiplication kernels.
# Each one uses a different matrix representation format.
# The first uses Compressed Sparse Row (CSR) format. In which only non-zero elements
# are stored in an array, and thier corresponding column and row index are stored in
# seperate 1D arrays.
#
# The Second uses padding and transposing, storing the matrix in ELL format.
# This second verson takes better advantage of memory bandwith inorder to reduce
# latency.However this comes with a cost of more memory usage.
# 
# Finaly a third hybrid implemntation is adviced wich comines idead from both 
# the CSR and ELL kernels.

In [4]:
%%cu
#include<iostream>

__global__ void sparse_matrix_csr_kernel(int num_rows, float* data,
                              int* col_index, int *row_ptr, float* x, float*y);

void print_array(float *array, int width);
int main(){
    int host_row_ptr [] = {0,2,2,5,7};
    int host_col_index [] = {0,2,1,2,3,0,3};
    float host_data[] = {3,1,2,4,1,1,1};
    float host_x[] = {2,4,6,1};
    float host_y[] = {0,0,0,0};

    int* device_row_ptr; int* device_col_index;
    float* device_data; float* device_x; float* device_y;
    
    cudaMalloc ((void **) &device_data, 7 * sizeof(float));
    cudaMalloc ((void **) &device_row_ptr, 5 * sizeof(float));
    cudaMalloc ((void **) &device_col_index, 7 * sizeof(float));
    cudaMalloc ((void **) &device_x, 4 * sizeof(float));
    cudaMalloc ((void **) &device_y, 4 * sizeof(float));
    
    cudaMemcpy(device_data,host_data, 7 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_row_ptr,host_row_ptr, 5 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_col_index,host_col_index, 7 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_x,host_x, 4 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_y,host_y, 4 *sizeof(float),cudaMemcpyHostToDevice);

    dim3 dim_grid(4);
    dim3 dim_block(4);

    sparse_matrix_csr_kernel<<<dim_grid,dim_block>>>(4,device_data,
                            device_col_index,device_row_ptr,device_x,device_y); 
  
    cudaMemcpy(host_y,device_y, 4 * sizeof(float), cudaMemcpyDeviceToHost);

    print_array(host_y,4);
    
    cudaFree(device_data);
    cudaFree(device_row_ptr);
    cudaFree(device_col_index);
    cudaFree(device_x);
    cudaFree(device_y);

    return 0;
}

void print_array(float *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");
}

__global__ void sparse_matrix_csr_kernel(int num_rows, float* data,
                              int* col_index, int *row_ptr, float* x, float*y)
{
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    int row_start = 0;
    int row_end = 0;
    float dot = 0;
    
    if (row < num_rows){
        dot = 0;
        row_start = row_ptr[row];
        row_end = row_ptr[row + 1];
        
        for (int elem = row_start; elem < row_end; elem++)
              dot += data[elem] * x[col_index[elem]];
        
        y[row] += dot;
    }
}

12	0	33	3	



In [19]:
%%cu
#include<iostream>

__global__ void sparse_matrix_vector_ell_kernel(int num_rows,float* data,int* col_index,
                                                int num_elem, float* x,float* y);
void print_array(float *array, int width);
int main(){
    float host_data[] = {3,0,2,1,1,0,4,1,0,0,1,0};
    int host_col_index [] = {0,0,1,0,2,0,2,3,0,0,3,0};
    float host_x[] = {2,4,6,1};
    float host_y[] = {0,0,0,0};

    int* device_col_index;
    float* device_data; float* device_x; float* device_y;
    
    cudaMalloc ((void **) &device_data, 7 * sizeof(float));
    cudaMalloc ((void **) &device_col_index, 7 * sizeof(float));
    cudaMalloc ((void **) &device_x, 4 * sizeof(float));
    cudaMalloc ((void **) &device_y, 4 * sizeof(float));
    
    cudaMemcpy(device_data,host_data, 7 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_col_index,host_col_index, 7 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_x,host_x, 4 * sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(device_y,host_y, 4 *sizeof(float),cudaMemcpyHostToDevice);

    dim3 dim_grid(2);
    dim3 dim_block(32);

    sparse_matrix_vector_ell_kernel<<<dim_grid,dim_block>>>(4,device_data,
                            device_col_index,3,device_x,device_y); 
  
    cudaMemcpy(host_y,device_y, 4 * sizeof(float), cudaMemcpyDeviceToHost);

    print_array(host_y,4);
    
    cudaFree(device_data);
    cudaFree(device_col_index);
    cudaFree(device_x);
    cudaFree(device_y);

    return 0;
}

void print_array(float *array, int width){
        for(int i = 0; i < width; i++)
        printf("%d\t", (int)array[i]);
    
  printf("\n");
}

__global__ void sparse_matrix_vector_ell_kernel(int num_rows,float* data,int* col_index,
                                                int num_elem, float* x,float* y)
{
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    float dot = 0;
    if (row < num_rows){
        dot = 0;
        for (int i = 0; i < num_elem;i++){
            dot += data[row + i * num_rows] * x[col_index[row+i*num_rows]];
        }
        y[row] += dot;
    }
}

12	0	32	2	

